In [7]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time
from tqdm import tqdm as ProgressBar
import six # needed for Google Cloud client
import sys
import pickle
import operator
import nltk
import zlib
import cPickle
import parsing_util
import cleaning_util

import en # NodeBox https://www.nodebox.net/code/index.php/Linguistics#verb_conjugation

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:

df=pd.read_pickle('CleanedIngredients.pkl')
ingredientsDict=pd.read_pickle('unique_ingredients.pkl')

In [3]:
#from parsing_util import Instructions
ingDict=parsing_util.Ingredients()
insDict=parsing_util.Instructions()   

In [17]:
# For debugging
reload(parsing_util)
url='http://allrecipes.com/recipe/143394/grilled-garlic-and-herb-shrimp/'
Instructions="[u'Whisk the paprika, garlic, Italian seasoning, lemon juice, olive oil, pepper, basil, and brown sugar together in a bowl until thoroughly blended. Stir in the shrimp, and toss to evenly coat with the marinade. Cover and refrigerate at least 2 hours, turning once.', u'Preheat an outdoor grill for medium-high heat. Lightly oil grill grate, and place about 4 inches from heat source.', u'Remove shrimp from marinade, drain excess, and discard marinade.', u'Place shrimp on preheated grill and cook, turning once, until opaque in the center, 5 to 6 minutes. Serve immediately.']"
ingredients=[u'2 teaspoons ground paprika', u'2 tablespoons fresh minced garlic', u'2 teaspoons Italian seasoning blend', u'2 tablespoons fresh lemon juice', u'1/4 cup olive oil', u'1/2 teaspoon ground black pepper', u'2 teaspoons dried basil leaves', u'2 tablespoons brown sugar, packed', u'2 pounds large shrimp (21-25 per pound), peeled and deveined', u'Add all ingredients to list', u'Add all ingredients to list']

parsedInstructions = insDict.parse(url, Instructions)

print Instructions
print ingredients
print parsing_util.get_ingredients(parsedInstructions, ingredients, ingredientsDict, 1)

[u'Whisk the paprika, garlic, Italian seasoning, lemon juice, olive oil, pepper, basil, and brown sugar together in a bowl until thoroughly blended. Stir in the shrimp, and toss to evenly coat with the marinade. Cover and refrigerate at least 2 hours, turning once.', u'Preheat an outdoor grill for medium-high heat. Lightly oil grill grate, and place about 4 inches from heat source.', u'Remove shrimp from marinade, drain excess, and discard marinade.', u'Place shrimp on preheated grill and cook, turning once, until opaque in the center, 5 to 6 minutes. Serve immediately.']
[u'2 teaspoons ground paprika', u'2 tablespoons fresh minced garlic', u'2 teaspoons Italian seasoning blend', u'2 tablespoons fresh lemon juice', u'1/4 cup olive oil', u'1/2 teaspoon ground black pepper', u'2 teaspoons dried basil leaves', u'2 tablespoons brown sugar, packed', u'2 pounds large shrimp (21-25 per pound), peeled and deveined', u'Add all ingredients to list', u'Add all ingredients to list']
set([u'olive o

In [ ]:
for idx, row in df.sample(3).iterrows():
    parsedInstructions = insDict.parse(row["url"], row["instructionSteps"])
    cleanIng = get_ingredients(parsedInstructions, row["ingredients"], ingredientsDict,2)
    print "url='"+row["url"]+"'"
    print 'Instructions="'+str(row["instructionSteps"])+'"'
    print 'ingredients='+str(row["ingredients"])
    # print cleanIng
    print
insDict.close()

In [19]:
# Count ingredients. prepare a clean list 
UniqueIngredientsDict=defaultdict(int)
cleanedIngredientsDict=pd.read_pickle('cleanerIngredients.pkl')  
#ingredientsDict=pd.read_pickle('unique_ingredients.pkl')

for url,ingredients in CleanIngredientsDict.items():
    for ingredient in ingredients:
        UniqueIngredientsDict[ingredient] += 1
    print item

#for url,ingredients in sorted(CleanIngredientsDict.items(), reverse=False, key=operator.itemgetter(1)):
    

set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'cayenne pepper', u'garlic', u'chile pepper', u'steak', u'kidney bean'])
set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'cayenne pepper', u'garlic', u'chile pepper', u'steak', u'kidney bean'])
set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'cayenne pepper', u'garlic', u'chile pepper', u'steak', u'kidney bean'])
set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'cayenne pepper', u'garlic', u'chile pepper', u'steak', u'kidney bean'])
set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'cayenne pepper', u'garlic', u'chile pepper', u'steak', u'kidney bean'])
set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'cayenne pepper', u'garlic', u'chile pepper', u'steak', u'kidney bean'])
set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'cayenne pepper', u'garlic', u'chile pepper', u'steak', u'kidney bean'])
set(['tomato', u'oil', u'onion', u'chili powder', u'cumin', u'

In [ ]:
print len(UniqueIngredientsDict)

In [ ]:
for item in sorted(UniqueIngredientsDict.items(), reverse=False, key=operator.itemgetter(1))[:30]:
    print item

In [ ]:
print df[['ingredients']].values[0]

In [ ]:
df["ingredients"].sample(5)

In [ ]:
df.sample(1)

In [ ]:
# Iterate through df parse ingredients into a compressed dictionary, and save on disk.
client = language.LanguageServiceClient()
IngredientsDict=defaultdict()
IngredientsDict=pd.read_pickle('parsedIngredients.pkl')  
added=0
#df2=df.sample(1)
for idx, row in ProgressBar(df.iterrows(), desc="Processing ingredients"):
    if row["url"] in IngredientsDict: continue
    # print 'loading ' + row["url"]   
    ingredients = '.'.join(ingredient for ingredient in row['ingredients'])
    if isinstance(ingredients, six.binary_type):
        ingredients = ingredients.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=ingredients,
        type=enums.Document.Type.PLAIN_TEXT)

        # Detects syntax in the document. You can also analyze HTML with:
        #   document.type == enums.Document.Type.HTML
    #tokens = client.analyze_syntax(document).tokens
    tokens = client.annotate_text(document, {'extract_syntax': True,}).tokens
    tokensList=list(tokens)
    IngredientsDict[row['url']]=zlib.compress(cPickle.dumps(tokensList))
    added += 1
    if added > 10000:
        with open('parsedIngredients.pkl', 'wb') as f:
            pickle.dump(IngredientsDict, f) 
        added = 0
    
with open('parsedIngredients.pkl', 'wb') as f:
    pickle.dump(IngredientsDict, f)
    



In [ ]:
IngredientsDict=pd.read_pickle('parsedIngredients.pkl')  
print len(IngredientsDict)
#for key,value in ingredientsDict.iteritems():
#    print key
#with open('parsedIngredients.pkl', 'wb') as f:
#    pickle.dump(IngredientsDict, f)

In [ ]:
# Iterate through df parse instructions into a compressed dictionary, and save on disk.
client = language.LanguageServiceClient()
InstructionsDict=defaultdict()
#InstructionsDict=pd.read_pickle('parsedInstruction.pkl')  
added=0

for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    if row["url"] in InstructionsDict: continue
    # print 'loading ' + row["url"]   
    instructions = '.'.join(instruction for instruction in row['instructionSteps'])
    if isinstance(instructions, six.binary_type):
        instructions = instructions.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=instructions,
        type=enums.Document.Type.PLAIN_TEXT)

        # Detects syntax in the document. You can also analyze HTML with:
        #   document.type == enums.Document.Type.HTML
    #tokens = client.analyze_syntax(document).tokens
    tokens = client.annotate_text(document, {'extract_syntax': True,}).tokens
    tokensList=list(tokens)
    InstructionsDict[row['url']]=zlib.compress(cPickle.dumps(tokensList))
    added += 1
    if added > 10000:
        with open('parsedInstruction.pkl', 'wb') as f:
            pickle.dump(InstructionsDict, f) 
        added = 0
    
with open('parsedInstruction.pkl', 'wb') as f:
    pickle.dump(InstructionsDict, f)
    
#ingredientsDict=pd.read_pickle('parsedInstruction.pkl')    
#for key,value in ingredientsDict.iteritems():
#    print key


In [ ]:
CleanIngredientsDict=pd.read_pickle('parsedInstruction.pkl')  
print len(IngredientsDict)
#for key,value in ingredientsDict.iteritems():
#    print key
#with open('parsedIngredients.pkl', 'wb') as f:
#    pickle.dump(IngredientsDict, f)

In [ ]:
# Iterate through df parse instructions into a compressed dictionary, and save on disk.
cleanedIngredientsDict=defaultdict()
cleanedIngredientsDict=pd.read_pickle('cleanerIngredients.pkl')  
added=0

for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    url=row["url"]
    if url in cleanedIngredientsDict: continue
    # print 'loading ' + row["url"]   
    parsedInstructions = insDict.parse(url, row["instructionSteps"])
    cleanIng = parsing_util.get_ingredients(parsedInstructions, row["ingredients"], ingredientsDict,0)
    cleanedIngredientsDict[url] = cleanIng
    added += 1
    if added > 10000:
        with open('cleanerIngredients.pkl', 'wb') as f:
            pickle.dump(cleanedIngredientsDict, f) 
        added = 0
    
with open('cleanerIngredients.pkl', 'wb') as f:
    pickle.dump(cleanedIngredientsDict, f)
    
#ingredientsDict=pd.read_pickle('parsedInstruction.pkl')    
#for key,value in ingredientsDict.iteritems():
#    print key


In [21]:
CleanIngredientsDict=pd.read_pickle('cleanerIngredients.pkl')  
print len(CleanIngredientsDict)
#for key,value in ingredientsDict.iteritems():
#    print key
#with open('parsedIngredients.pkl', 'wb') as f:
#    pickle.dump(IngredientsDict, f)
for key,item in sorted(CleanIngredientsDict.items(), reverse=False, key=operator.itemgetter(1))[:5]:
    print item

45245
set([u'onion', u'ginger paste', u'soy sauce', u'rice wine vinegar', u'lemon juice', u'black pepper', u'garlic', u'yogurt', u'sugar', u'ketchup'])
set([u'olive oil', u'pepper', u'pink salt', u'celery', u'chicken', u'kosher salt'])
set([u'ginger ale', u'pineapple juice', u'white grape juice'])
set([u'whiskey', u'lime soda', u'amaretto'])
set([u'red bell pepper', u'tea', 'almond', u'sesame seed', u'water', u'cabbage', u'rice', u'salt', u'edamame'])


In [ ]:
Ingredients=defaultDict()
idx=0
df2=df.sample(10)
for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    # print 'Ingredients:',row['cleanedIngredients']
    if type(row['instructionSteps']) != float:
        instructions = '.'.join(instruction for instruction in row['instructionSteps'])
        #print row['name']
        #print instructions
        #print row['ingredients']
        #print row['cleanedIngredients']
        ingredients = row['cleanedIngredients']
        oldIngredientsSet |=set(ingredients)
        # ingredients = ' '.join(ingredient for ingredient in row['cleanedIngredients'])
        ingredients = get_ingredients(instructions, ingredients, ingredientsDict)
        ingredientsSet |= ingredients
#    if idx==100:
#        break
print idx
print len(ingredientsSet)
print len(oldIngredientsSet)

In [ ]:
InstructionsDict=pd.read_pickle('parsedInstruction.pkl')    

#for key,value in InstructionsDict.iteritems():
#    print key

#with open('parsedIngredients.pkl', 'wb') as f:
#    pickle.dump(InstructionsDict, f)

In [ ]:
print ingredientsSet
print oldIngredientsSet
with open('ingredientsSet.pkl', 'wb') as handle:
    pickle.dump(ingredientsSet, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
verbVertor =  [get_ingredients('.'.join(df[['instructionSteps']].values[i][0]),
                               df['cleanedIngredients'].values[i],
                               ingredientsDict
                              ) for i in range(df.shape[0])]
df['cleanerIngredients']=verbVertor

In [ ]:
# Store data (serialize to pickle file)
df.to_pickle('cleanerIngredients.pkl')

In [ ]:
en.noun.singular('abalone shells')

In [ ]:
print en.noun.singular('8 ounces pancetta bacon diced')
print en.noun.singular('pancetta bacon')

In [ ]:
print en.noun.plural('chily')
print en.noun.plural('chili')

In [ ]:
from cleaning_util import extract_ingredients
ingredients=[u'1 cup packed brown sugar', u'1 cup water', u'1 cup raisins', u'2 tablespoons butter', u'1/2 teaspoon salt', u'1 1/2 cups whole heat flour', u'3/4 teaspoon baking soda', u'1/2 teaspoon ground ginger', u'1 teaspoon ground cinnamon', u'1/2 cup dates, pitted and chopped', u'1/2 cup candied mixed fruit peel, chopped', u'1/2 cup chopped nuts', u'1/2 cup chopped dried mixed fruit', u'Add all ingredients to list', u'Add all ingredients to list']
extract_ingredients(ingredients)